In [0]:
dbutils.widgets.text("proc_date", "")
proc_date = dbutils.widgets.get("proc_date")

In [0]:
%run ../../../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
create_temp_view_with_clean_columns(
    spark,
    catalog_name,
    'udp_wcm_bronze_sap_bw',
    'sap_bw_wcm_vcdmdmm10',
    proc_date,
    'temp_sap_assortment_site_active'
)

In [0]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {catalog_name}.udp_wcm_silver_sap_bw.sap_assortment_site_active
(
  file_creation_ts TIMESTAMP,
  hash_id BIGINT,
  store_id STRING,
  assortment STRING,
  proc_date DATE
)
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
""")

In [0]:
spark.sql(f"""
CREATE OR REPLACE TEMP VIEW sap_assortment_site_active
AS
SELECT * EXCEPT (rn) FROM (
	SELECT
		FARM_FINGERPRINT(CONCAT(
								{catalog_name}.default.fn_CLEAN_SAP_STRING1((CAST(SITE AS STRING), "")),
								{catalog_name}.default.fn_CLEAN_SAP_STRING1((CAST(FILIA AS STRING), "")),
								ifnull(CAST(proc_date AS STRING), "")
							)) AS hash_id,
		ROW_NUMBER() OVER (PARTITION BY site, filia ORDER BY proc_date DESC) rn,
		{catalog_name}.default.fn_CLEAN_SAP_STRING1(site) store_id,
		{catalog_name}.default.fn_CLEAN_SAP_STRING1(filia) assortment,
		proc_date,
  		file_creation_ts
	FROM temp_sap_assortment_site_active WHERE {catalog_name}.default.fn_clean_sap_string(filia) IS NOT NULL 
) WHERE rn = 1;
""")


In [0]:
spark.sql(f"""
MERGE INTO {catalog_name}.udp_wcm_silver_sap_bw.sap_assortment_site_active inf_tbl
USING sap_assortment_site_active tmp_tbl
ON inf_tbl.hash_id = tmp_tbl.hash_id
WHEN NOT MATCHED THEN INSERT *;
	""")